# Valuing Caps and Floors

We value caps and floors using Black's model

In [11]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
from financepy.finutils import *
from financepy.products.libor import *

####################################################################
# FINANCEPY BETA Version 0.180 - This build:  22 Sep 2020 at 22:40 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#    For bug reports and comments - allmypythonprojects@gmail.com  #
####################################################################



## Building a Libor Curve

In [2]:
valuationDate = FinDate(6, 6, 2018)

In [3]:
spotDays = 0
settlementDate = valuationDate
depoDCCType = FinDayCountTypes.THIRTY_E_360_ISDA

depos = []
depo = FinLiborDeposit(settlementDate, "1M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "2M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "3M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "6M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "9M", 0.0230, depoDCCType); depos.append(depo)

In [5]:
accrual = FinDayCountTypes.THIRTY_E_360
freq = FinFrequencyTypes.SEMI_ANNUAL
longEnd = FinDateGenRuleTypes.BACKWARD
swapType = FinLiborSwapTypes.PAYER

spotDays = 2
settlementDate = valuationDate.addWorkDays(spotDays)

swaps = []
swap = FinLiborSwap(settlementDate, "1Y", swapType, 0.0250, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "2Y", swapType, 0.0255, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "3Y", swapType, 0.0260, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "4Y", swapType, 0.0265, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "5Y", swapType, 0.0270, freq, accrual); swaps.append(swap)

In [28]:
liborCurve = FinLiborCurve(valuationDate, depos, [], swaps)

## Creating a Cap and Floor

In [29]:
capType = FinLiborCapFloorTypes.CAP
floorType = FinLiborCapFloorTypes.FLOOR

In [30]:
strikeRate = 0.02

In [31]:
cap = FinLiborCapFloor(settlementDate, "2Y", capType, strikeRate)
flr = FinLiborCapFloor(settlementDate, "2Y", floorType, strikeRate)

In [32]:
print(cap)

START DATE: FRI 08 JUN 2018
MATURITY DATE: MON 08 JUN 2020
STRIKE COUPON: 2.0
OPTION TYPE: FinLiborCapFloorTypes.CAP
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA


In [33]:
print(flr)

START DATE: FRI 08 JUN 2018
MATURITY DATE: MON 08 JUN 2020
STRIKE COUPON: 2.0
OPTION TYPE: FinLiborCapFloorTypes.FLOOR
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA


# Valuation

### Black's Model

We start with Black's model with 25% volatility

In [34]:
model = FinModelBlack(0.25)

In [35]:
cap.value(valuationDate, liborCurve, model)

11361.05137142763

In [36]:
flr.value(valuationDate, liborCurve, model)

826.5171694471896

In [37]:
cap.printLeg()

START DATE: FRI 08 JUN 2018
MATURITY DATE: MON 08 JUN 2020
OPTION TYPE FinLiborCapFloorTypes.CAP
STRIKE (%): 2.0
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE WED 06 JUN 2018
PAYMENT_DATE     YEAR_FRAC   FWD_RATE    INTRINSIC           DF    CAPLET_PV       CUM_PV
FRI 08 JUN 2018          -         -            -     1.000000            -            -
MON 10 SEP 2018  0.2555556   2.29838       757.99     0.994034       757.99       757.99
MON 10 DEC 2018  0.2500000   2.28739       710.13     0.988382       759.95      1517.94
FRI 08 MAR 2019  0.2444444   2.29062       698.24     0.982878       815.40      2333.34
MON 10 JUN 2019  0.2555556   3.08483      2703.56     0.975190      2714.71      5048.05
MON 09 SEP 2019  0.2472222   2.58278      1396.11     0.969003      1504.89      6552.94
MON 09 DEC 2019  0.2500000   2.55409      1333.76     0.962855      1493.33      8046.28
MON 09 MAR 2020  0.2500000   2.60448      1445.65     0.95

# Alternative Models

## Shifted Black

Shifted Black gives the same pdf at F+S as Black does at F. So if we want to have the PDF for F=0.25 at -0.25 because rates are negative then you need to set F=-0.50.

In [38]:
model = FinModelBlackShifted(0.25, -0.005)

In [39]:
cap.value(valuationDate, liborCurve, model)

11889.637179252899

In [40]:
flr.value(valuationDate, liborCurve, model)

1355.106744260354

The floor has increased in value as the downside risk is greater.

## SABR Model

In [41]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573

In [42]:
model = FinModelSABR(alpha, beta, rho, nu)

In [43]:
cap.value(valuationDate, liborCurve, model)

11462.605315193625

In [44]:
flr.value(valuationDate, liborCurve, model)

928.0711132131839

## Shifted SABR Model

In [45]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573; shift = -0.005

In [46]:
model = FinModelSABRShifted(alpha, beta, rho, nu, shift)

In [47]:
cap.value(valuationDate, liborCurve, model)

11828.173009456626

In [48]:
flr.value(valuationDate, liborCurve, model)

1293.6388074761853

## Hull White Model

In [49]:
sigma = 0.01; alpha = 0.005

In [50]:
model = FinModelRatesHW(sigma, alpha)

In [51]:
cap.value(valuationDate, liborCurve, model)

14282.947550977811

In [52]:
flr.value(valuationDate, liborCurve, model)

2765.4505366363196

Copyright (c) 2020 Dominic O'Kane